<a href="https://colab.research.google.com/github/Dobenkss/Data_Manipulation/blob/main/DataManipulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Mahasiswa
Nama: Abhipraya Rasendriya Atma Ardany<br />
Nim: 24.21.1587<br />
Kelas: IF08<br />

#Installasi PySpark

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=0694d886cee465ea409b2bb01e1af1e5b3ea9afc5a59abadf7e4010ad2bbe6af
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


#Start Spark Session

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
print(spark)
print(spark.version)

3.5.3


#Import Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/My Drive/Big Data/Pertemuan 2/flights_small.csv'
flights = spark.read.csv(file_path, header=True, inferSchema=True)

flights.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

#Filtering Data

Lakukan filtering data pada dataset dengan mencari nilai kurang dari 1000 dari feature arr_time, dan tunjukkan hasilnya

In [ ]:
filter_flights = flights.filter(flights.arr_time < 1000)

filter_flights.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|   11| 12|    2346|       -4|     217|      -28|     AS| N765AS|   121|   SEA| ANC|     183|    1448|  23|    46|
|2014|    1| 13|    2156|       -9|     607|      -15|     AS| N597AS|    24|   SEA| BOS|     290|    2496|  21|    56|
|2014|   10| 29|     643|        3|     851|        2|     OO| N218AG|  3456|   PDX| BUR|     108|     817|   6|    43|
|2014|    2| 25|     555|       -5|     819|       -3|     AS| N549AS|   604|   SEA| LAS|     121|     867|   5|    55|
|2014|   12| 14|     633|       -7|     

#Select Data

Lakukan select data dengan feature carrier, tailnum, dan distance. Filter A berdasarkan feature carrier yang memiliki value "AS", dan filter B menggunakan feature dest yang memiliki value "SEA". Munculkan hasil dari proses filter

In [ ]:
selectData = flights.select('carrier','tailnum','distance', 'dest')

filterA = flights.carrier == "AS"
filterB = flights.dest == "SEA"

selected = selectData.filter(filterA).filter(filterB)
print(selected.show())

+-------+-------+--------+----+
|carrier|tailnum|distance|dest|
+-------+-------+--------+----+
|     AS| N791AS|     129| SEA|
|     AS| N320AS|     129| SEA|
|     AS| N799AS|     129| SEA|
|     AS| N760AS|     129| SEA|
|     AS| N520AS|     129| SEA|
|     AS| N556AS|     129| SEA|
|     AS| N705AS|     129| SEA|
|     AS| N796AS|     129| SEA|
|     AS| N536AS|     129| SEA|
|     AS| N529AS|     129| SEA|
+-------+-------+--------+----+

None


#Data Agregation

Lakukan proses agregasi data dengan memunculkan count, mean, standard deviasi, minimal value, dan max value. Gunakan feature dep_delay, dan arr_delay

In [ ]:
flights = flights.withColumn('dep_delay', flights.dep_delay.cast('float'))
flights = flights.withColumn('arr_delay', flights.arr_delay.cast('float'))

flights.describe('arr_delay', 'dep_delay').show()

+-------+------------------+------------------+
|summary|         arr_delay|         dep_delay|
+-------+------------------+------------------+
|  count|              9925|              9952|
|   mean|2.2530982367758186| 6.068629421221865|
| stddev|31.074918600451877|28.808608062751805|
|    min|             -58.0|             -19.0|
|    max|             900.0|             886.0|
+-------+------------------+------------------+



#Grouping & Aggregation

Lakukan grouping dan aggregation menggunakan feature month, dan dest untuk menghitung rata-rata nilai dari feature arr_delay

In [ ]:
import pyspark.sql.functions as F

by_month_dest = flights.groupBy('month', 'dest')

flights = flights.withColumn('arr_delay', flights.arr_delay.cast('float'))

by_month_dest.avg('arr_delay').show()

+-----+----+-------------------+
|month|dest|     avg(arr_delay)|
+-----+----+-------------------+
|    4| PHX|-3.1333333333333333|
|    1| RDM|             -1.625|
|    5| ONT|  3.111111111111111|
|    7| OMA|               -9.0|
|    8| MDW|               6.55|
|    6| DEN|               -1.0|
|    5| IAD|-13.444444444444445|
|   12| COS|                7.0|
|   11| ANC| -5.235294117647059|
|    5| AUS|               -3.5|
|    5| COS| 15.666666666666666|
|    2| PSP|                5.8|
|    4| ORD| -4.560975609756097|
|   10| DFW| 14.515151515151516|
|   10| DCA|              -13.0|
|    8| JNU|              17.75|
|   11| KOA|               -1.8|
|   10| OMA|               -3.0|
|    6| ONT|                3.0|
|    3| MSP|              -2.95|
+-----+----+-------------------+
only showing top 20 rows

